# Chapter 2: BLEU and ROUGE

**Hands-on implementation of n-gram metrics for evaluating generated text.**

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand n-grams** - The fundamental building blocks of text comparison used by BLEU and ROUGE
2. **Implement BLEU from scratch** - Including modified precision, geometric mean, and brevity penalty
3. **Implement ROUGE from scratch** - Including ROUGE-N and ROUGE-L variants
4. **Recognize gaming vulnerabilities** - How naive metrics can be exploited and how BLEU/ROUGE defend against this
5. **Apply these metrics in practice** - Using the `evaluate` library for production use cases

## Why These Metrics Matter

When we build systems that generate text (machine translation, summarization, chatbots), we need ways to automatically measure quality. Human evaluation is the gold standard, but it's expensive and slow.

**BLEU** (Bilingual Evaluation Understudy) and **ROUGE** (Recall-Oriented Understudy for Gisting Evaluation) are the foundational automatic metrics that have been used for decades:

- **BLEU** (2002): Originally designed for machine translation, asks "How much of the machine output is correct?"
- **ROUGE** (2004): Originally designed for summarization, asks "How much of the reference content is captured?"

These metrics aren't perfect, but understanding them deeply teaches you:
- How to think about text similarity mathematically
- The tradeoffs between precision and recall
- How to design metrics that resist gaming

Let's build them from scratch!

---

## Part 1: Building BLEU from Scratch

We'll start by importing the basic tools we need. BLEU is fundamentally about counting and comparing word sequences, so we only need `Counter` for counting and `math` for logarithms.

In [ ]:
from collections import Counter
import math

---

## N-grams: The Building Blocks

An **n-gram** is a contiguous sequence of n tokens (usually words) from a text. N-grams are the fundamental unit of comparison in both BLEU and ROUGE.

| N | Name | Example from "the cat sat" |
|---|------|---------------------------|
| 1 | Unigram | "the", "cat", "sat" |
| 2 | Bigram | "the cat", "cat sat" |
| 3 | Trigram | "the cat sat" |

**Why n-grams?** They capture local word order and phrase structure:
- **Unigrams** tell us if the right words are present (vocabulary coverage)
- **Bigrams** tell us if word pairs are correct (basic grammar/fluency)
- **Higher n-grams** capture longer phrases (more sophisticated structure)

The key insight: if a translation has the right unigrams but wrong bigrams, it likely has the right words in the wrong order!

Let's implement n-gram extraction:

In [ ]:
def get_ngrams(tokens: list[str], n: int) -> list[tuple]:
    """Extract n-grams from a list of tokens."""
    return [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]

def tokenize(text: str) -> list[str]:
    """Simple whitespace tokenizer with lowercasing."""
    return text.lower().split()

# Example from the book
sentence = "The cat is on the mat"
tokens = tokenize(sentence)

print(f"Tokens: {tokens}")
print(f"Unigrams (n=1): {get_ngrams(tokens, 1)}")
print(f"Bigrams (n=2): {get_ngrams(tokens, 2)}")
print(f"Trigrams (n=3): {get_ngrams(tokens, 3)}")

Notice how n-grams slide over the text like a window. A sentence with $m$ tokens produces $m - n + 1$ n-grams of size $n$. This means:
- Longer texts produce more n-grams
- Shorter texts may produce zero n-grams for higher values of n (you can't get a 5-gram from a 3-word sentence)

---

## The Gaming Problem: Why Naive Precision Fails

Before we build BLEU properly, let's understand *why* it was designed the way it was. 

**Precision** measures: "What fraction of my output is correct?"

$$\text{Precision} = \frac{\text{Words in candidate that appear in reference}}{\text{Total words in candidate}}$$

This sounds reasonable, but it can be easily gamed. Consider a pathological translation that just repeats a common word:

In [ ]:
# Example 2 from the book
candidate = "the the the the the the the"
reference1 = "The cat is on the mat."
reference2 = "There is a cat on the mat."

cand_tokens = tokenize(candidate)
ref1_tokens = tokenize(reference1)
ref2_tokens = tokenize(reference2)

# Naive precision: what fraction of candidate words appear in references?
ref_words = set(ref1_tokens) | set(ref2_tokens)
matches = sum(1 for w in cand_tokens if w in ref_words)
naive_precision = matches / len(cand_tokens)

print(f"Candidate: '{candidate}'")
print(f"Naive precision: {matches}/{len(cand_tokens)} = {naive_precision:.0%}")
print("\n^ This is nonsense but scores 100%!")

This is absurd! A nonsense output of just "the the the..." scores 100% precision because "the" appears in the references.

**The lesson:** Any metric that can be gamed this easily is useless for evaluation. We need a smarter approach.

---

## Modified N-gram Precision: The First Defense

BLEU's solution is elegant: **clip each n-gram count by its maximum occurrence in any reference**.

The intuition: If "the" appears at most 2 times in any reference, then we should only credit at most 2 occurrences of "the" in the candidate, no matter how many times it appears.

$$\text{Modified Precision}_n = \frac{\sum_{\text{n-gram}} \min(\text{Count}_{\text{candidate}}, \text{MaxCount}_{\text{reference}})}{\sum_{\text{n-gram}} \text{Count}_{\text{candidate}}}$$

This completely defeats the repetition attack:

In [ ]:
def modified_precision(candidate: str, references: list[str], n: int = 1) -> float:
    """
    Calculate modified n-gram precision.
    Clips counts by max occurrence in any single reference.
    """
    cand_ngrams = get_ngrams(tokenize(candidate), n)
    cand_counts = Counter(cand_ngrams)
    
    # Get max count for each n-gram across all references
    max_ref_counts = Counter()
    for ref in references:
        ref_counts = Counter(get_ngrams(tokenize(ref), n))
        for ngram, count in ref_counts.items():
            max_ref_counts[ngram] = max(max_ref_counts[ngram], count)
    
    # Clip candidate counts
    clipped_count = sum(
        min(count, max_ref_counts.get(ngram, 0))
        for ngram, count in cand_counts.items()
    )
    total_count = sum(cand_counts.values())
    
    return clipped_count / total_count if total_count > 0 else 0

# Test on the gaming example
references = [reference1, reference2]
mod_precision = modified_precision(candidate, references, n=1)

print(f"Modified precision: {mod_precision:.2f}")
print(f"'the' appears max 2 times in any reference")
print(f"So: 2/7 = {2/7:.2f}")

Now the gaming attempt gets a much more appropriate score of 0.29 instead of 1.00.

**Key insight:** By capping counts at the maximum reference occurrence, we ensure that repeating words provides diminishing returns. You can't score higher than what the reference actually contains.

---

## Real Translation Evaluation

Let's apply modified precision to a realistic machine translation example. Here we have a Chinese sentence with three human reference translations, and two candidate machine translations of varying quality.

**Chinese source:** 「它是保证军队永远听党指挥的行动指南。」

This example demonstrates how n-gram precision at different levels captures different aspects of translation quality:

In [ ]:
# Example 1 from the book
candidate1 = "It is a guide to action which ensures that the military always obeys the commands of the party."
candidate2 = "It is to ensure the troops forever hearing the activity guidebook that party direct."

ref1 = "It is a guide to action that ensures that the military will forever heed Party commands."
ref2 = "It is the guiding principle which guarantees the military forces always being under the command of the Party."
ref3 = "It is the practical guide for the army always to heed the directions of the party."

references = [ref1, ref2, ref3]

# Calculate precision for different n-gram levels
print("N-gram precisions:")
print("-" * 40)
for n in range(1, 5):
    p1 = modified_precision(candidate1, references, n)
    p2 = modified_precision(candidate2, references, n)
    print(f"{n}-gram: Candidate1={p1:.3f}, Candidate2={p2:.3f}")

**Observations:**
- Candidate 1 (good) maintains high precision even at higher n-gram levels
- Candidate 2 (poor) has decent unigram precision (it uses some right words) but crashes at higher levels (wrong word order, bad phrasing)

This is exactly what we want! The metric correctly identifies that Candidate 2 has structural problems even though it contains some correct vocabulary.

---

## Combining N-gram Levels: Why Geometric Mean?

We have precision scores at multiple n-gram levels (1-gram, 2-gram, 3-gram, 4-gram). How do we combine them into a single score?

BLEU uses the **geometric mean** rather than the arithmetic mean. The key property is the **zero-product rule**: if any component is zero, the entire product is zero.

$$\text{Geometric Mean} = \sqrt[n]{p_1 \times p_2 \times \cdots \times p_n}$$

**Why does this matter?** It penalizes systems that completely fail at any n-gram level:

In [ ]:
def arithmetic_mean(values):
    return sum(values) / len(values)

def geometric_mean(values):
    product = 1
    for v in values:
        product *= v
    return product ** (1 / len(values))

# System A: Balanced performance
system_a = [0.80, 0.60, 0.40, 0.20]

# System B: Degenerate - only unigrams work
system_b = [0.80, 0.00, 0.00, 0.00]

print("System A (balanced): p1=0.80, p2=0.60, p3=0.40, p4=0.20")
print(f"  Arithmetic mean: {arithmetic_mean(system_a):.3f}")
print(f"  Geometric mean:  {geometric_mean(system_a):.3f}")

print("\nSystem B (degenerate): p1=0.80, p2=0.00, p3=0.00, p4=0.00")
print(f"  Arithmetic mean: {arithmetic_mean(system_b):.3f}")
print(f"  Geometric mean:  {geometric_mean(system_b):.3f}")
print("\n^ Geometric mean = 0 if ANY component fails!")

**The zero-product property is a feature, not a bug!**

System B has zero bigram/trigram/4-gram precision, meaning it produces no correct word sequences at all. With arithmetic mean, it would still score 0.20 - hiding a catastrophic failure.

Geometric mean correctly gives it a score of 0, signaling that the system is fundamentally broken despite getting some unigrams right.

---

## Brevity Penalty: The Second Defense

Modified precision solved the repetition problem, but there's another gaming strategy: **output only high-confidence words**.

If a system outputs just "the military party" (3 words), every word might be correct (100% precision), but it's clearly not a complete translation. We need to penalize outputs that are too short.

**Brevity Penalty (BP):**

$$BP = \begin{cases} 1 & \text{if } c \geq r \\ e^{(1 - r/c)} & \text{if } c < r \end{cases}$$

Where $c$ = candidate length, $r$ = reference length.

The penalty is exponential: the shorter you are, the more severe the penalty.

In [ ]:
def brevity_penalty(candidate_len: int, reference_len: int) -> float:
    """Calculate BLEU brevity penalty."""
    if candidate_len >= reference_len:
        return 1.0
    return math.exp(1 - reference_len / candidate_len)

# Gaming example: very short candidate
candidate_short = "the military party"  # 3 words
reference = ref1  # 16 words

c_len = len(tokenize(candidate_short))
r_len = len(tokenize(reference))

bp = brevity_penalty(c_len, r_len)
precision = modified_precision(candidate_short, [reference], n=1)

print(f"Candidate: '{candidate_short}' ({c_len} words)")
print(f"Reference length: {r_len} words")
print(f"\nUnigram precision: {precision:.2f} (looks perfect!)")
print(f"Brevity penalty: {bp:.4f}")
print(f"Adjusted score: {precision * bp:.4f}")

**The brevity penalty in action:** Even with perfect precision, a 3-word output compared to a 16-word reference gets crushed by a penalty of ~0.01. This makes the gaming strategy ineffective.

**Why exponential?** Linear penalties would be too gentle. A translation that's half the length should be penalized severely, not just get a 50% reduction.

---

## Complete BLEU Implementation

Now we can put it all together. The complete BLEU formula is:

$$\text{BLEU} = BP \times \exp\left(\sum_{n=1}^{N} w_n \log p_n\right)$$

Where:
- $BP$ = Brevity Penalty
- $p_n$ = Modified precision for n-grams of size n
- $w_n$ = Weight for each n-gram level (typically uniform: $w_n = 1/N$)
- $N$ = Maximum n-gram size (typically 4)

The log-sum formulation is equivalent to geometric mean but numerically more stable.

**Standard BLEU-4** uses n-grams from 1 to 4 with equal weights (0.25 each).

In [ ]:
def bleu_score(candidate: str, references: list[str], max_n: int = 4) -> dict:
    """
    Calculate BLEU score with breakdown.
    """
    cand_tokens = tokenize(candidate)
    
    # Find closest reference length
    ref_lens = [len(tokenize(ref)) for ref in references]
    closest_ref_len = min(ref_lens, key=lambda r: abs(r - len(cand_tokens)))
    
    # Calculate precisions
    precisions = []
    for n in range(1, max_n + 1):
        p = modified_precision(candidate, references, n)
        precisions.append(p)
    
    # Geometric mean (with smoothing for zeros)
    epsilon = 1e-10
    log_precisions = [math.log(max(p, epsilon)) for p in precisions]
    geo_mean = math.exp(sum(log_precisions) / len(log_precisions))
    
    # Brevity penalty
    bp = brevity_penalty(len(cand_tokens), closest_ref_len)
    
    return {
        "bleu": bp * geo_mean,
        "brevity_penalty": bp,
        "precisions": precisions,
        "geometric_mean": geo_mean,
    }

# Calculate BLEU for both candidates
result1 = bleu_score(candidate1, references)
result2 = bleu_score(candidate2, references)

print("Candidate 1 (good translation):")
print(f"  Precisions: {[f'{p:.3f}' for p in result1['precisions']]}")
print(f"  BLEU: {result1['bleu']:.3f}")

print("\nCandidate 2 (poor translation):")
print(f"  Precisions: {[f'{p:.3f}' for p in result2['precisions']]}")
print(f"  BLEU: {result2['bleu']:.3f}")

The BLEU scores reflect what we'd expect:
- **Candidate 1** (the good translation) scores substantially higher
- **Candidate 2** (the poor translation) is penalized for its bad n-gram precision at higher levels

**Interpreting BLEU scores:**
- BLEU is typically reported as a score between 0 and 1 (or 0-100 as a percentage)
- Scores above 0.40 generally indicate good translations
- Scores above 0.60 are excellent
- Human translations typically score around 0.60-0.80 against each other (there's natural variation)

---

## Part 2: Building ROUGE from Scratch

While BLEU focuses on **precision** (how much of the candidate is correct), ROUGE focuses on **recall** (how much of the reference is covered).

| Metric | Primary Question | Best For |
|--------|------------------|----------|
| BLEU | "Is the output correct?" | Machine Translation |
| ROUGE | "Is the reference covered?" | Summarization |

**Why recall for summarization?** A summary should capture the key information from the source. We care more about "Did the summary include the important points?" than "Is every word in the summary justified?"

ROUGE computes:
- **Recall:** What fraction of reference n-grams appear in the candidate?
- **Precision:** What fraction of candidate n-grams appear in the reference?
- **F1:** Harmonic mean of precision and recall

Typically, ROUGE-1 recall (or ROUGE-L F1) is the primary metric reported.

In [ ]:
def rouge_n(candidate: str, reference: str, n: int = 1) -> dict:
    """
    Calculate ROUGE-N recall, precision, and F1.
    """
    cand_ngrams = Counter(get_ngrams(tokenize(candidate), n))
    ref_ngrams = Counter(get_ngrams(tokenize(reference), n))
    
    # Count matches (clipped)
    matches = sum(
        min(cand_ngrams[ng], ref_ngrams[ng])
        for ng in ref_ngrams
    )
    
    recall = matches / sum(ref_ngrams.values()) if ref_ngrams else 0
    precision = matches / sum(cand_ngrams.values()) if cand_ngrams else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {"recall": recall, "precision": precision, "f1": f1}

# Example from the book
reference = "Google announced new AI features for search."
candidate = "Google revealed AI search capabilities."

r1 = rouge_n(candidate, reference, n=1)
r2 = rouge_n(candidate, reference, n=2)

print(f"Reference: '{reference}'")
print(f"Candidate: '{candidate}'")
print(f"\nROUGE-1: recall={r1['recall']:.3f}, precision={r1['precision']:.3f}, F1={r1['f1']:.3f}")
print(f"ROUGE-2: recall={r2['recall']:.3f}, precision={r2['precision']:.3f}, F1={r2['f1']:.3f}")

**Observations:**
- **ROUGE-1** (unigrams): Measures vocabulary overlap. The candidate captures about 43% of the reference's words.
- **ROUGE-2** (bigrams): Measures phrase overlap. Only about 14% of reference bigrams appear in the candidate.

The lower ROUGE-2 score indicates that while the candidate uses similar vocabulary, the phrasing is quite different from the reference. This is common with paraphrases.

**Why F1?** In summarization, we often care about both precision and recall. F1 gives a balanced view:
$$F1 = \frac{2 \times \text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$$

---

## ROUGE-L: Longest Common Subsequence

ROUGE-N requires exact contiguous matches, but sometimes good text has the right words in roughly the right order without being exactly contiguous.

**ROUGE-L** uses the **Longest Common Subsequence (LCS)** algorithm:
- A subsequence doesn't need to be contiguous, just in the same order
- Automatically rewards longer in-sequence matches
- More flexible than strict n-gram matching

**Example:**
- Reference: "The quick brown fox"
- Candidate: "The brown quick fox"
- LCS: "The", "brown", "fox" (length 3) - "quick" is out of order

The LCS algorithm uses dynamic programming to efficiently find the longest matching subsequence:

In [ ]:
def lcs_length(x: list, y: list) -> int:
    """Calculate length of longest common subsequence."""
    m, n = len(x), len(y)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if x[i-1] == y[j-1]:
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])
    
    return dp[m][n]

def rouge_l(candidate: str, reference: str) -> dict:
    """Calculate ROUGE-L using longest common subsequence."""
    cand_tokens = tokenize(candidate)
    ref_tokens = tokenize(reference)
    
    lcs = lcs_length(ref_tokens, cand_tokens)
    
    recall = lcs / len(ref_tokens) if ref_tokens else 0
    precision = lcs / len(cand_tokens) if cand_tokens else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {"lcs": lcs, "recall": recall, "precision": precision, "f1": f1}

# Example from the book
reference = "The company announced strong quarterly earnings"
candidate = "Strong earnings were announced by the company"

result = rouge_l(candidate, reference)
print(f"Reference: '{reference}'")
print(f"Candidate: '{candidate}'")
print(f"\nLCS length: {result['lcs']}")
print(f"ROUGE-L recall: {result['recall']:.3f}")

**Observations:**
- The candidate is a paraphrase (passive voice, reordered)
- ROUGE-L finds an LCS of 4 words that appear in both, in order
- This gives a recall of 0.67 - the candidate captures most of the reference content despite different phrasing

**When to use ROUGE-L vs ROUGE-N:**
- **ROUGE-N:** When exact phrasing matters (news summaries, factual text)
- **ROUGE-L:** When you want flexibility for paraphrases and different orderings

---

## Part 3: Using Production Implementations

Our from-scratch implementations help us understand the algorithms, but in practice you should use established libraries like Hugging Face's `evaluate`.

**Why use library implementations?**
- Thoroughly tested and validated
- Handle edge cases (empty strings, unicode, etc.)
- Optimized for performance
- Consistent with published results

The `evaluate` library provides standardized implementations of BLEU, ROUGE, and many other metrics:

In [ ]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# BLEU expects: predictions (list of str), references (list of list of str)
bleu_result = bleu.compute(
    predictions=[candidate1],
    references=[[ref1, ref2, ref3]]
)

# ROUGE expects: predictions, references (both list of str)
rouge_result = rouge.compute(
    predictions=["Google revealed AI search capabilities."],
    references=["Google announced new AI features for search."]
)

print("BLEU (evaluate library):")
print(f"  Score: {bleu_result['bleu']:.3f}")
print(f"  Precisions: {[f'{p:.3f}' for p in bleu_result['precisions']]}")

print("\nROUGE (evaluate library):")
for key, value in rouge_result.items():
    print(f"  {key}: {value:.3f}")

**Note the API differences:**
- **BLEU:** Expects `references` as a list of lists (each prediction can have multiple references)
- **ROUGE:** Expects `references` as a list of strings (one reference per prediction)

The library also provides additional variants like `rougeLsum` (sentence-level ROUGE-L for multi-sentence summaries).

---

## Exercises

Test your understanding of BLEU and ROUGE with these exercises.

### Exercise 1: Brevity Penalty in Action

What happens when we try to game BLEU with a very short but "correct" output?

In [ ]:
# Exercise 1
result = bleu_score("party", ["The military follows party commands"])
print(f"BLEU: {result['bleu']:.4f}")
print(f"Brevity penalty crushes the score: {result['brevity_penalty']:.4f}")

### Exercise 2: ROUGE-2 and Paraphrasing

Why does ROUGE-2 often return 0 for short paraphrased texts?

*Think about it:* If two sentences use the same words but in different order, what happens to bigram overlap?

### Exercise 3: Implement ROUGE-S (Skip-bigram)

ROUGE-S allows gaps between words in bigrams. Try implementing it for the ROUGE-L example sentence pair above.

---

## Summary and Key Takeaways

### What We Learned

**N-grams** are the foundation of traditional text evaluation:
- Unigrams capture vocabulary, higher n-grams capture phrase structure
- Comparing n-gram overlap gives a quantitative similarity measure

**BLEU** (Precision-oriented, for Machine Translation):
- Modified precision clips counts to prevent gaming by repetition
- Geometric mean ensures failure at any n-gram level tanks the score
- Brevity penalty prevents gaming by outputting only confident words
- Combine all n-gram levels (typically 1-4) for a holistic score

**ROUGE** (Recall-oriented, for Summarization):
- ROUGE-N measures recall of n-grams (how much reference content is covered)
- ROUGE-L uses longest common subsequence for flexible matching
- Reports precision, recall, and F1 (typically F1 is primary metric)

### Strengths of These Metrics

1. **Fast and deterministic** - No model inference needed
2. **Interpretable** - Based on clear counting operations
3. **Language-agnostic** - Work on any language with word boundaries
4. **Well-established** - Decades of research and correlation studies

### Limitations to Be Aware Of

1. **Ignore semantics** - "The dog bit the man" vs "The man bit the dog" may score similarly
2. **Require references** - Can't evaluate open-ended generation
3. **Penalize valid paraphrases** - Correct but differently-worded text scores lower
4. **Correlation varies by domain** - BLEU correlates well with human judgment for MT, less so for other tasks

### When to Use What

| Task | Primary Metric | Why |
|------|----------------|-----|
| Machine Translation | BLEU | Precision matters; multiple references available |
| Summarization | ROUGE-1/2/L | Recall matters; need to cover source content |
| General Text Gen | Consider alternatives | BLEU/ROUGE may not capture quality well |

### Next Steps

In Chapter 3, we'll explore **semantic metrics** like BERTScore and COMET that use neural networks to compare meaning rather than exact word overlap. These address the key limitation of n-gram metrics: they can recognize that "The automobile was repaired" and "The car was fixed" are semantically equivalent!